In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('all_stocks_5yr.csv',parse_dates=True, index_col='date')

In [3]:
df.head()

,open,high,low,close,volume,Name
date,,,,,,
2013-02-08,15.07,15.12,14.63,14.75,8407500,AAL
2013-02-11,14.89,15.01,14.26,14.46,8882000,AAL
2013-02-12,14.45,14.51,14.10,14.27,8126000,AAL
2013-02-13,14.30,14.94,14.25,14.66,10259500,AAL
2013-02-14,14.94,14.96,13.16,13.99,31879900,AAL


In [5]:
df = df[['Name','close']]

In [21]:
df_grp = df.groupby('Name').mean()

In [22]:
df_grp = df_grp.reset_index()

In [23]:
df_grp = df_grp.sort_values(by=['close'],ascending = False)
df_grp = df_grp.reset_index(drop=True)

In [83]:
df_grp.head()

,Name,close
0,PCLN,1312.873538
1,GOOG,725.403353
2,GOOGL,682.233847
3,AZO,619.703654
4,AMZN,576.880041


In [27]:
top_50 = list(df_grp['Name'][:50])

In [80]:
top_50[:5]

['PCLN', 'GOOG', 'GOOGL', 'AZO', 'AMZN']

In [29]:
len(top_50)

50

In [30]:
temp = [df, df_grp]

In [36]:
top_50_dfnames = list()
for x in top_50:
    top_50_dfnames.append('df_'+x)

In [37]:
top_50_dfnames[:5]

['df_PCLN', 'df_GOOG', 'df_GOOGL', 'df_AZO', 'df_AMZN']

In [53]:
#List of dataframes of top 50 stocks of S&P 500
top_50_dfs = list()
for i in range(50):
    top_50_dfnames[i] = df[df['Name']==top_50[i]]
    top_50_dfs.append(top_50_dfnames[i])

In [54]:
len(top_50_dfs)

50

In [72]:
#Dictionary of dataframes of top 50 stocks of S&P 500
top_50_dfs_dic = {}
for i,j in zip(top_50_dfnames,top_50_dfs):
    top_50_dfs_dic[i] = j

TypeError: unhashable type: 'DataFrame'

In [69]:
type(top_50_dfs[2])

pandas.core.frame.DataFrame

In [74]:
from pmdarima import auto_arima
from math import sqrt
from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_squared_error
import warnings
warnings.filterwarnings("ignore")

In [89]:
#ARIMA Model

def ARIMA_model(df):
    stepwise_fit = auto_arima(df['close'], start_p=1, start_q=1,
                      test='adf',
                      max_p=9, max_q=3,
                      m=1,             
                      d=1, max_d=3,          
                      seasonal=False,   
                      start_P=0, 
                      D=None, 
                      trace=False,
                      error_action='ignore',  
                      suppress_warnings=True, 
                      stepwise=False)
    order = stepwise_fit.get_params()['order']
    n = int(len(df) * 0.8)
    train = df['close'][:n]
    test = df['close'][n:]
    model = ARIMA(train.values, order = order)
    result = model.fit()
    res = pd.Series(result.forecast(len(test)))
    res_df = res.to_frame(name='predictions')
    res_df = res_df.reset_index(drop=True)
    test_df = test.to_frame(name='Actual')
    test_df = test_df.reset_index()
    eval_df = pd.concat([test_df,res_df],axis=1)
    eval_df = eval_df.set_index('date', drop=True)
    root_mse = sqrt(mean_squared_error(eval_df['Actual'],eval_df['predictions']))
    
    return root_mse

In [91]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

In [92]:
#Linear Regression
def Linear_model(df):
    def lagit(df, lags):
        names = list()
        for i in range(1,lags+1):
            df['lag_'+str(i)] = df['close'].shift(i)
            names.append('lag_'+str(i))
        df.dropna(inplace=True)    
        return names
    
    lagnames = lagit(df,5)
    X = np.array(df[lagnames])
    Y = np.array(df['close'])
    x_train,x_test,y_train,y_test = train_test_split(X,Y, test_size=0.2, shuffle=False)
    lr = LinearRegression().fit(x_train,y_train)
    pred = lr.predict(x_test)
    n = len(y_test)
    eval_df = pd.DataFrame(df['close'][-n:])
    eval_df = eval_df.reset_index()
    pred_df = pd.Series(pred).to_frame(name='Predictions')
    pred_df = pred_df.reset_index(drop=True)
    eval_df = pd.concat([eval_df,pred_df],axis=1)
    eval_df = eval_df.set_index('date', drop=True)
    root_mse = sqrt(mean_squared_error(eval_df['close'],eval_df['Predictions']))
    
    return root_mse

In [97]:
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import LSTM, Dense

In [98]:
#LSTM
def LSTM_model(df):
    df['returns'] = df.close.pct_change()
    df['log_returns'] = np.log(1 + df['returns'])
    df.dropna(inplace = True)
    X = df[['close','log_returns']].values
    scaler = MinMaxScaler(feature_range =(0,1)).fit(X)
    X_scaled = scaler.transform(X)
    Y = [x[0] for x in X_scaled]
    split = int(len(X_scaled) * 0.8)
    X_train = X_scaled[:split]
    X_test = X_scaled[split:len(X_scaled)]
    Y_train = Y[:split]
    Y_test = Y[split:len(Y)] 
    n = 3
    Xtrain = list()
    Ytrain = list()
    Xtest = list()
    Ytest = list()
    for i in range(n,len(X_train)):
        Xtrain.append(X_train[i-n:i,:X_train.shape[1]])
        Ytrain.append(Y_train[i])
    for i in range(n,len(X_test)):
        Xtest.append(X_test[i-n:i,:X_test.shape[1]])
        Ytest.append(Y_test[i])
    Xtrain , Ytrain = (np.array(Xtrain),np.array(Ytrain))
    Xtrain = np.reshape(Xtrain, (Xtrain.shape[0], Xtrain.shape[1], Xtrain.shape[2]))

    Xtest, Ytest = (np.array(Xtest),np.array(Ytest))
    Xtest = np.reshape(Xtest, (Xtest.shape[0], Xtest.shape[1], Xtest.shape[2]))
    model = Sequential()
    model.add(LSTM(4, input_shape=(Xtrain.shape[1],Xtrain.shape[2])))
    model.add(Dense(1))
    model.compile(loss="mean_squared_error", optimizer="adam")
    model.fit(Xtrain, Ytrain, epochs=100, validation_data=(Xtest, Ytest), batch_size = 16, verbose=1)
    trainPredict = model.predict(Xtrain)
    testPredict = model.predict(Xtest)
    trainPredict = np.c_[trainPredict, np.zeros(trainPredict.shape)]
    testPredict = np.c_[testPredict, np.zeros(testPredict.shape)]
    trainPredict = scaler.inverse_transform(trainPredict)
    trainPredict = [x[0] for x in trainPredict]

    testPredict = scaler.inverse_transform(testPredict)
    testPredict = [x[0] for x in testPredict]
    z = len(Ytest)
    eval_df = pd.DataFrame(df['close'][-z:])
    eval_df = eval_df.reset_index()
    pred_df = pd.Series(testPredict).to_frame(name='Predictions')
    pred_df = pred_df.reset_index(drop=True)
    eval_df = pd.concat([eval_df,pred_df],axis=1)
    eval_df = eval_df.set_index('date', drop=True)
    root_mse = sqrt(mean_squared_error(eval_df['close'],eval_df['Predictions']))
    
    return root_mse

In [99]:
temp = LSTM_model(top_50_dfs[2])

Epoch 1/100
63/63 [==============================] - 2s 7ms/step - loss: 0.1204 - val_loss: 0.5617
Epoch 2/100
63/63 [==============================] - 0s 2ms/step - loss: 0.0506 - val_loss: 0.3420
Epoch 3/100
63/63 [==============================] - 0s 2ms/step - loss: 0.0286 - val_loss: 0.2417
Epoch 4/100
63/63 [==============================] - 0s 2ms/step - loss: 0.0242 - val_loss: 0.2050
Epoch 5/100
63/63 [==============================] - 0s 2ms/step - loss: 0.0207 - val_loss: 0.1744
Epoch 6/100
63/63 [==============================] - 0s 2ms/step - loss: 0.0171 - val_loss: 0.1457
Epoch 7/100
63/63 [==============================] - 0s 2ms/step - loss: 0.0135 - val_loss: 0.1171
Epoch 8/100
63/63 [==============================] - 0s 2ms/step - loss: 0.0097 - val_loss: 0.0816
Epoch 9/100
63/63 [==============================] - 0s 2ms/step - loss: 0.0061 - val_loss: 0.0554
Epoch 10/100
63/63 [==============================] - 0s 2ms/step - loss: 0.0032 - val_loss: 0.0327
Epoch 11/

63/63 [==============================] - 0s 2ms/step - loss: 1.4779e-04 - val_loss: 0.0076
Epoch 81/100
63/63 [==============================] - 0s 2ms/step - loss: 1.4842e-04 - val_loss: 0.0082
Epoch 82/100
63/63 [==============================] - 0s 2ms/step - loss: 1.4990e-04 - val_loss: 0.0077
Epoch 83/100
63/63 [==============================] - 0s 2ms/step - loss: 1.4812e-04 - val_loss: 0.0076
Epoch 84/100
63/63 [==============================] - 0s 2ms/step - loss: 1.4791e-04 - val_loss: 0.0083
Epoch 85/100
63/63 [==============================] - 0s 2ms/step - loss: 1.4771e-04 - val_loss: 0.0087
Epoch 86/100
63/63 [==============================] - 0s 2ms/step - loss: 1.4963e-04 - val_loss: 0.0080
Epoch 87/100
63/63 [==============================] - 0s 2ms/step - loss: 1.4710e-04 - val_loss: 0.0072
Epoch 88/100
63/63 [==============================] - 0s 2ms/step - loss: 1.5426e-04 - val_loss: 0.0086
Epoch 89/100
63/63 [==============================] - 0s 2ms/step - loss: 1.4

In [100]:
temp

68.04301836790263

In [101]:
ARIMA_error = list()
LR_error = list()

for x in top_50_dfs:
    ar_err = ARIMA_model(x)
    ARIMA_error.append(ar_err)
    lr_err = Linear_model(x)
    LR_error.append(lr_err)

In [106]:
LSTM_error = list()

for x in top_50_dfs:
    lstm_err = LSTM_model(x)
    LSTM_error.append(lstm_err)

Epoch 1/100
63/63 [==============================] - 2s 11ms/step - loss: 0.0489 - val_loss: 0.2413
Epoch 2/100
63/63 [==============================] - 0s 4ms/step - loss: 0.0156 - val_loss: 0.1620
Epoch 3/100
63/63 [==============================] - 0s 5ms/step - loss: 0.0114 - val_loss: 0.1044
Epoch 4/100
63/63 [==============================] - 0s 4ms/step - loss: 0.0077 - val_loss: 0.0606
Epoch 5/100
63/63 [==============================] - 0s 4ms/step - loss: 0.0046 - val_loss: 0.0269
Epoch 6/100
63/63 [==============================] - 0s 4ms/step - loss: 0.0023 - val_loss: 0.0072
Epoch 7/100
63/63 [==============================] - 0s 4ms/step - loss: 9.9090e-04 - val_loss: 0.0011
Epoch 8/100
63/63 [==============================] - 0s 4ms/step - loss: 5.3232e-04 - val_loss: 4.9938e-04
Epoch 9/100
63/63 [==============================] - 0s 4ms/step - loss: 4.2041e-04 - val_loss: 9.2256e-04
Epoch 10/100
63/63 [==============================] - 0s 4ms/step - loss: 3.9945e-04 - v

63/63 [==============================] - 0s 4ms/step - loss: 2.2855e-04 - val_loss: 0.0020
Epoch 80/100
63/63 [==============================] - 0s 4ms/step - loss: 2.2822e-04 - val_loss: 0.0015
Epoch 81/100
63/63 [==============================] - 0s 4ms/step - loss: 2.2889e-04 - val_loss: 0.0024
Epoch 82/100
63/63 [==============================] - 0s 4ms/step - loss: 2.4013e-04 - val_loss: 0.0015
Epoch 83/100
63/63 [==============================] - 0s 4ms/step - loss: 2.3502e-04 - val_loss: 0.0022
Epoch 84/100
63/63 [==============================] - 0s 4ms/step - loss: 2.3359e-04 - val_loss: 0.0017
Epoch 85/100
63/63 [==============================] - 0s 4ms/step - loss: 2.2483e-04 - val_loss: 0.0017
Epoch 86/100
63/63 [==============================] - 0s 4ms/step - loss: 2.2957e-04 - val_loss: 0.0016
Epoch 87/100
63/63 [==============================] - 0s 4ms/step - loss: 2.2402e-04 - val_loss: 8.0979e-04
Epoch 88/100
63/63 [==============================] - 0s 4ms/step - loss:

49/49 [==============================] - 0s 2ms/step - loss: 2.1374e-04 - val_loss: 4.7884e-04
Epoch 56/100
49/49 [==============================] - 0s 2ms/step - loss: 2.1165e-04 - val_loss: 5.6350e-04
Epoch 57/100
49/49 [==============================] - 0s 2ms/step - loss: 2.1173e-04 - val_loss: 5.2215e-04
Epoch 58/100
49/49 [==============================] - 0s 2ms/step - loss: 2.0796e-04 - val_loss: 4.3052e-04
Epoch 59/100
49/49 [==============================] - 0s 2ms/step - loss: 2.1545e-04 - val_loss: 5.1334e-04
Epoch 60/100
49/49 [==============================] - 0s 2ms/step - loss: 2.1113e-04 - val_loss: 5.5064e-04
Epoch 61/100
49/49 [==============================] - 0s 2ms/step - loss: 2.1158e-04 - val_loss: 6.4187e-04
Epoch 62/100
49/49 [==============================] - 0s 2ms/step - loss: 2.0687e-04 - val_loss: 6.1782e-04
Epoch 63/100
49/49 [==============================] - 0s 2ms/step - loss: 2.0551e-04 - val_loss: 6.6698e-04
Epoch 64/100
49/49 [=====================

63/63 [==============================] - 0s 2ms/step - loss: 1.5620e-04 - val_loss: 0.0073
Epoch 32/100
63/63 [==============================] - 0s 2ms/step - loss: 1.5442e-04 - val_loss: 0.0069
Epoch 33/100
63/63 [==============================] - 0s 2ms/step - loss: 1.5106e-04 - val_loss: 0.0068
Epoch 34/100
63/63 [==============================] - 0s 2ms/step - loss: 1.5219e-04 - val_loss: 0.0070
Epoch 35/100
63/63 [==============================] - 0s 2ms/step - loss: 1.5051e-04 - val_loss: 0.0071
Epoch 36/100
63/63 [==============================] - 0s 2ms/step - loss: 1.5230e-04 - val_loss: 0.0070
Epoch 37/100
63/63 [==============================] - 0s 2ms/step - loss: 1.4892e-04 - val_loss: 0.0070
Epoch 38/100
63/63 [==============================] - 0s 2ms/step - loss: 1.4969e-04 - val_loss: 0.0074
Epoch 39/100
63/63 [==============================] - 0s 2ms/step - loss: 1.5103e-04 - val_loss: 0.0070
Epoch 40/100
63/63 [==============================] - 0s 2ms/step - loss: 1.5

63/63 [==============================] - 0s 2ms/step - loss: 0.0027 - val_loss: 8.5407e-04
Epoch 10/100
63/63 [==============================] - 0s 2ms/step - loss: 7.1918e-04 - val_loss: 6.8684e-04
Epoch 11/100
63/63 [==============================] - 0s 2ms/step - loss: 3.6916e-04 - val_loss: 6.9894e-04
Epoch 12/100
63/63 [==============================] - 0s 2ms/step - loss: 3.3033e-04 - val_loss: 7.0912e-04
Epoch 13/100
63/63 [==============================] - 0s 2ms/step - loss: 3.2560e-04 - val_loss: 7.1793e-04
Epoch 14/100
63/63 [==============================] - 0s 2ms/step - loss: 3.2755e-04 - val_loss: 7.1510e-04
Epoch 15/100
63/63 [==============================] - 0s 2ms/step - loss: 3.3041e-04 - val_loss: 7.1162e-04
Epoch 16/100
63/63 [==============================] - 0s 2ms/step - loss: 3.2284e-04 - val_loss: 7.0487e-04
Epoch 17/100
63/63 [==============================] - 0s 2ms/step - loss: 3.1775e-04 - val_loss: 7.0257e-04
Epoch 18/100
63/63 [=========================

...

63/63 [==============================] - 0s 2ms/step - loss: 6.2200e-04 - val_loss: 0.0030
Epoch 24/100
63/63 [==============================] - 0s 2ms/step - loss: 6.2062e-04 - val_loss: 0.0032
Epoch 25/100
63/63 [==============================] - 0s 2ms/step - loss: 5.9682e-04 - val_loss: 0.0031
Epoch 26/100
63/63 [==============================] - 0s 2ms/step - loss: 5.8634e-04 - val_loss: 0.0030
Epoch 27/100
63/63 [==============================] - 0s 2ms/step - loss: 5.7273e-04 - val_loss: 0.0031
Epoch 28/100
63/63 [==============================] - 0s 2ms/step - loss: 5.6296e-04 - val_loss: 0.0030
Epoch 29/100
63/63 [==============================] - 0s 2ms/step - loss: 5.4808e-04 - val_loss: 0.0029
Epoch 30/100
63/63 [==============================] - 0s 2ms/step - loss: 5.3644e-04 - val_loss: 0.0028
Epoch 31/100
63/63 [==============================] - 0s 2ms/step - loss: 5.2361e-04 - val_loss: 0.0029
Epoch 32/100
63/63 [==============================] - 0s 2ms/step - loss: 5.1

8/8 [==============================] - 0s 1ms/step
Epoch 1/100
63/63 [==============================] - 2s 7ms/step - loss: 0.0954 - val_loss: 0.2317
Epoch 2/100
63/63 [==============================] - 0s 2ms/step - loss: 0.0153 - val_loss: 0.0860
Epoch 3/100
63/63 [==============================] - 0s 2ms/step - loss: 0.0084 - val_loss: 0.0721
Epoch 4/100
63/63 [==============================] - 0s 2ms/step - loss: 0.0075 - val_loss: 0.0611
Epoch 5/100
63/63 [==============================] - 0s 2ms/step - loss: 0.0066 - val_loss: 0.0508
Epoch 6/100
63/63 [==============================] - 0s 2ms/step - loss: 0.0057 - val_loss: 0.0421
Epoch 7/100
63/63 [==============================] - 0s 2ms/step - loss: 0.0047 - val_loss: 0.0334
Epoch 8/100
63/63 [==============================] - 0s 2ms/step - loss: 0.0038 - val_loss: 0.0225
Epoch 9/100
63/63 [==============================] - 0s 2ms/step - loss: 0.0029 - val_loss: 0.0154
Epoch 10/100
63/63 [==============================] - 0s 2

Epoch 77/100
63/63 [==============================] - 0s 2ms/step - loss: 1.3100e-04 - val_loss: 5.5604e-04
Epoch 78/100
63/63 [==============================] - 0s 2ms/step - loss: 1.3361e-04 - val_loss: 4.7226e-04
Epoch 79/100
63/63 [==============================] - 0s 2ms/step - loss: 1.2720e-04 - val_loss: 4.9101e-04
Epoch 80/100
63/63 [==============================] - 0s 2ms/step - loss: 1.2732e-04 - val_loss: 5.1360e-04
Epoch 81/100
63/63 [==============================] - 0s 2ms/step - loss: 1.3490e-04 - val_loss: 5.8711e-04
Epoch 82/100
63/63 [==============================] - 0s 2ms/step - loss: 1.3007e-04 - val_loss: 7.7246e-04
Epoch 83/100
63/63 [==============================] - 0s 2ms/step - loss: 1.2891e-04 - val_loss: 8.0552e-04
Epoch 84/100
63/63 [==============================] - 0s 2ms/step - loss: 1.3026e-04 - val_loss: 5.4714e-04
Epoch 85/100
63/63 [==============================] - 0s 2ms/step - loss: 1.2833e-04 - val_loss: 7.9893e-04
Epoch 86/100
63/63 [========

63/63 [==============================] - 0s 2ms/step - loss: 1.1566e-04 - val_loss: 0.0030
Epoch 55/100
63/63 [==============================] - 0s 2ms/step - loss: 1.1849e-04 - val_loss: 0.0036
Epoch 56/100
63/63 [==============================] - 0s 2ms/step - loss: 1.2096e-04 - val_loss: 0.0030
Epoch 57/100
63/63 [==============================] - 0s 2ms/step - loss: 1.1967e-04 - val_loss: 0.0033
Epoch 58/100
63/63 [==============================] - 0s 2ms/step - loss: 1.1893e-04 - val_loss: 0.0033
Epoch 59/100
63/63 [==============================] - 0s 2ms/step - loss: 1.1404e-04 - val_loss: 0.0031
Epoch 60/100
63/63 [==============================] - 0s 2ms/step - loss: 1.1326e-04 - val_loss: 0.0029
Epoch 61/100
63/63 [==============================] - 0s 2ms/step - loss: 1.1471e-04 - val_loss: 0.0026
Epoch 62/100
63/63 [==============================] - 0s 2ms/step - loss: 1.1486e-04 - val_loss: 0.0029
Epoch 63/100
63/63 [==============================] - 0s 2ms/step - loss: 1.1

63/63 [==============================] - 0s 2ms/step - loss: 2.9689e-04 - val_loss: 4.1799e-04
Epoch 32/100
63/63 [==============================] - 0s 2ms/step - loss: 2.9992e-04 - val_loss: 4.3316e-04
Epoch 33/100
63/63 [==============================] - 0s 2ms/step - loss: 2.9686e-04 - val_loss: 4.0330e-04
Epoch 34/100
63/63 [==============================] - 0s 2ms/step - loss: 2.8837e-04 - val_loss: 4.0500e-04
Epoch 35/100
63/63 [==============================] - 0s 2ms/step - loss: 2.8699e-04 - val_loss: 3.9244e-04
Epoch 36/100
63/63 [==============================] - 0s 2ms/step - loss: 2.8327e-04 - val_loss: 3.8882e-04
Epoch 37/100
63/63 [==============================] - 0s 2ms/step - loss: 2.8696e-04 - val_loss: 3.9333e-04
Epoch 38/100
63/63 [==============================] - 0s 2ms/step - loss: 2.8448e-04 - val_loss: 3.8489e-04
Epoch 39/100
63/63 [==============================] - 0s 2ms/step - loss: 2.8938e-04 - val_loss: 4.0212e-04
Epoch 40/100
63/63 [=====================

In [107]:
len(LSTM_error)

50

In [108]:
final_result = pd.DataFrame({'Ticks':top_50,
                            'ARIMA_error':ARIMA_error,
                            'LR_error':LR_error,
                            'LSTM_error':LSTM_error})

In [109]:
final_result.head()

,Ticks,ARIMA_error,LR_error,LSTM_error
0,PCLN,269.117250,26.430802,45.168493
1,GOOG,115.026549,11.871150,16.283746
2,GOOGL,166.094100,11.158808,55.929940
3,AZO,117.012554,10.715571,10.974645
4,AMZN,250.747781,14.951085,35.611937


In [110]:
final_result.to_csv('top_50.csv', index=False)